## Subgraph Classification with Average Pooling - Variable Misuse

The pooling function is made of average Pooling

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

[16:59:05] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory
Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "10_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [4]:
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Variable misuse    35036
Correct            35036
Name: label, dtype: int64

dev
Variable misuse    3754
Correct            3754
Name: label, dtype: int64

eval
Variable misuse    18787
Correct            18787
Name: label, dtype: int64



# Config

In [5]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=0,
    epochs=10, 
    
    train_frac=0.8,
    random_seed=42, 
    
    # model parameters
    elem_emb_size=300,
    node_emb_size=300,                  # *** dimensionality of node embeddings
    h_dim=300,                           # *** should match to node dimensionality
    n_layers=5,
    dropout=0.1,
    activation="tanh"
)

In [6]:
save_config(config, "average_pooling_var_misuse_subgraph.yaml")

# Create Dataset

In [7]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [8]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [9]:
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective

class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClfAveragePooling",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

## Launch Tensorboard

In [10]:
# %tensorboard --logdir './10_percent_v1'

## Start model training and Evaluation

In [11]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 4900346


2022-09-12 17:32:40.093561: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Epoch 1:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 0, Time: 2695 s
{'Accuracy/test/VariableMisuseClfAveragePooling_': 0.6111364188532555,
 'Accuracy/train/VariableMisuseClfAveragePooling_': 0.6071428571428571,
 'Accuracy/train_avg/VariableMisuseClfAveragePooling': 0.5177451283889468,
 'Accuracy/val/VariableMisuseClfAveragePooling_': 0.610800040355125,
 'Loss/test/VariableMisuseClfAveragePooling_': 0.6522616470346645,
 'Loss/train/VariableMisuseClfAveragePooling_': 0.662276566028595,
 'Loss/train_avg/VariableMisuseClfAveragePooling': 0.690386695692139,
 'Loss/val/VariableMisuseClfAveragePooling_': 0.6507278953568411,
 'grad_norm/train/_': 0.25459479826013315}


Epoch 2:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 1, Time: 2675 s
{'Accuracy/test/VariableMisuseClfAveragePooling_': 0.6857135264820213,
 'Accuracy/train/VariableMisuseClfAveragePooling_': 0.5892857142857143,
 'Accuracy/train_avg/VariableMisuseClfAveragePooling': 0.6671052202815433,
 'Accuracy/val/VariableMisuseClfAveragePooling_': 0.6823105831315577,
 'Loss/test/VariableMisuseClfAveragePooling_': 0.5733780143212299,
 'Loss/train/VariableMisuseClfAveragePooling_': 0.6589162945747375,
 'Loss/train_avg/VariableMisuseClfAveragePooling': 0.5980922325792974,
 'Loss/val/VariableMisuseClfAveragePooling_': 0.571388602761899,
 'grad_norm/train/_': 0.6910436913873927}


...

Epoch 9:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 8, Time: 3476 s
{'Accuracy/test/VariableMisuseClfAveragePooling_': 0.7014751882896015,
 'Accuracy/train/VariableMisuseClfAveragePooling_': 0.75,
 'Accuracy/train_avg/VariableMisuseClfAveragePooling': 0.7561872718978102,
 'Accuracy/val/VariableMisuseClfAveragePooling_': 0.7042852098466506,
 'Loss/test/VariableMisuseClfAveragePooling_': 0.5802235402623002,
 'Loss/train/VariableMisuseClfAveragePooling_': 0.49755287170410156,
 'Loss/train_avg/VariableMisuseClfAveragePooling': 0.4790137867531637,
 'Loss/val/VariableMisuseClfAveragePooling_': 0.5822743318848691,
 'grad_norm/train/_': 0.8798314150006676}


Epoch 9: 100%|████████████████████████████████| 548/548 [38:12<00:00,  4.18s/it]


Epoch: 9, Time: 3468 s
{'Accuracy/test/VariableMisuseClfAveragePooling_': 0.7159355259961127,
 'Accuracy/train/VariableMisuseClfAveragePooling_': 0.75,
 'Accuracy/train_avg/VariableMisuseClfAveragePooling': 0.7591098312043796,
 'Accuracy/val/VariableMisuseClfAveragePooling_': 0.7158242534301856,
 'Loss/test/VariableMisuseClfAveragePooling_': 0.5779261351848135,
 'Loss/train/VariableMisuseClfAveragePooling_': 0.48964256048202515,
 'Loss/train_avg/VariableMisuseClfAveragePooling': 0.47425653094792886,
 'Loss/val/VariableMisuseClfAveragePooling_': 0.577930888382055,
 'grad_norm/train/_': 1.6477083356174802}
Final eval: Loss/val/VariableMisuseClfAveragePooling_final: 0.577930888382055, Accuracy/val/VariableMisuseClfAveragePooling_final: 0.7158242534301856, Loss/test/VariableMisuseClfAveragePooling_final: 0.5779261355902873, Accuracy/test/VariableMisuseClfAveragePooling_final: 0.7159355259961127


(<__main__.Trainer at 0x7f0c1bad2e80>,
 {'Loss/val/VariableMisuseClfAveragePooling_final': 0.577930888382055,
  'Accuracy/val/VariableMisuseClfAveragePooling_final': 0.7158242534301856,
  'Loss/test/VariableMisuseClfAveragePooling_final': 0.5779261355902873,
  'Accuracy/test/VariableMisuseClfAveragePooling_final': 0.7159355259961127})

In [12]:
!nvidia-smi

Tue Sep 13 02:53:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0    71W / 300W |  19938MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------